In [37]:
#05.29 01:30ver.

#validation_split 0.8
#model = isensee_2017

#write_data_to_file will be executed in the final step, with config["data_file"]
#fetch_data_to_file 대신 Naomi코드 사용

In [38]:
import tensorflow as tf
print(tf.__version__)

for i in tf.config.list_physical_devices():
    print(i)

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

#for multi-gpu usage
#mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

#for mixed-precision training
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

2.2.0
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')
Compute dtype: float16
Variable dtype: float32


As crop_img and crop_img_to functions are missing in utils.py, we deactivated crop-related functions.

In [39]:
#unet3d/utils/sitk_utils.py
#여기는 전처리 부분, 복셀간 원점, 좌표, 스페이싱, 방향, 보간법, transform, data <--> image 등을 하는 함수들을 정의

import SimpleITK as sitk
import numpy as np

#복셀 원점 좌표 보정
def calculate_origin_offset(new_spacing, old_spacing):
    return np.subtract(new_spacing, old_spacing)/2


#복셀간 공간 재조정(이미지, 스페이싱 (1, 1, 1), 선형보간법 사용, 기본값 = 0)
#즉, 복셀간 공간을 재조정하여, resample_to_spacing 함수에 파라미터를 넘겨주고, 호출하는 함수
def sitk_resample_to_spacing(image, new_spacing=(1.0, 1.0, 1.0), interpolator=sitk.sitkLinear, default_value=0.):
    #줌팩터 = 기존 이미지로부터 얻은 공간간의 간격/새이미지로 부터 얻은 공간간의 간격
    zoom_factor = np.divide(image.GetSpacing(), new_spacing)
    #새로운 사이즈 = np.asarray(올림(반올림(줌팩터 * 이미지로부터 얻은 사이즈)), 소수점 다섯째짜리까지)
    new_size = np.asarray(np.ceil(np.round(np.multiply(zoom_factor, image.GetSize()), decimals=5)), dtype=np.int16)
    #오프셋 = (새로 만들 이미지의 복셀 간격, 이미지로부터 얻은 복셀 간격)
    offset = calculate_origin_offset(new_spacing, image.GetSpacing())
    #참조 이미지 = 빈이미지(새 사이즈, 새 복셀간 공간, 이미지 방향, 원점+(보정값), 디폴트값)
    reference_image = sitk_new_blank_image(size=new_size, spacing=new_spacing, direction=image.GetDirection(),
                                           origin=image.GetOrigin() + offset, default_value=default_value)
    #이미지 재조정(이미지, 참조이미지, 보간법, 디폴트값)
    return sitk_resample_to_image(image, reference_image, interpolator=interpolator, default_value=default_value)


#이미지 재조정(이미지, 참조된 이미지, 디폴트값, 선형보간법, transform=None, output_pixel_type=None)
def sitk_resample_to_image(image, reference_image, default_value=0., interpolator=sitk.sitkLinear, transform=None,
                           output_pixel_type=None):
    #transform에 아무값도 안넣어주면, sitk에 있는 Transform 사용 후, 항등행렬로 변환
    if transform is None:
        transform = sitk.Transform()
        transform.SetIdentity()
    #output_pixel 타입에 아무값도 안 넣어주면, 이미지의 GetPixelID()값(픽셀타입을 얻어오는함수)을 output_pixel_type에 대입하여 사용
    if output_pixel_type is None:
        output_pixel_type = image.GetPixelID()
    resample_filter = sitk.ResampleImageFilter() #resample_filter에 sitkResampleImageFilter 적용
    resample_filter.SetInterpolator(interpolator) #선형보간법 설정
    resample_filter.SetTransform(transform) #transform 설정
    resample_filter.SetOutputPixelType(output_pixel_type) #픽셀타입설정
    resample_filter.SetDefaultPixelValue(default_value) #픽셀값 디폴트값으로 설정
    resample_filter.SetReferenceImage(reference_image) #참조된 이미지 값에 적용
    return resample_filter.Execute(image) #이미지 재조정 필터 실행


#빈 복셀이미지 형성(사이즈, 복셀간의 공간, 방향, 원점, 디폴트값=0)
def sitk_new_blank_image(size, spacing, direction, origin, default_value=0.):
    #이미지 = 1로 구성된, 이미지배열을 받아, 전치행렬을 구한후, default값을 곱해서, 이미지 Array를 받아 image에 저장
    image = sitk.GetImageFromArray(np.ones(size, dtype=np.float).T * default_value)
    #복셀 이미지의 스페이싱, 다이렉션, 원점 정의
    image.SetSpacing(spacing)
    image.SetDirection(direction)
    image.SetOrigin(origin)
    return image

#공간 재조정(데이터, 공간, 타겟으로 하는 공간, 선형보간법, 디폴트값)
def resample_to_spacing(data, spacing, target_spacing, interpolation="linear", default_value=0.):
    #다음 함수를 실행해, image에 저장, 데이터를 이미지로 변환시켜, image에 저장
    image = data_to_sitk_image(data, spacing=spacing)
    #만약, 선형보간법을 사용할 경우
    if interpolation is "linear":
        interpolator = sitk.sitkLinear
    #만약, 최단입점보간법을 사용할 경우(즉, output을 만들때, input에서 가장 가까운 놈(비슷한 놈)을 가져다 쓴다.)
    elif interpolation is "nearest":
        interpolator = sitk.sitkNearestNeighbor
    #그외 값들은 에러처리
    else:
        raise ValueError("'interpolation' must be either 'linear' or 'nearest'. '{}' is not recognized".format(
            interpolation))
    resampled_image = sitk_resample_to_spacing(image, new_spacing=target_spacing, interpolator=interpolator,
                                               default_value=default_value)
    return sitk_image_to_data(resampled_image)


#data -> sitk_image
def data_to_sitk_image(data, spacing=(1., 1., 1.)):
    #3차원이면, 다음과 같이 회전
    if len(data.shape) == 3:
        data = np.rot90(data, 1, axes=(0, 2))
    #data array를 받아, image에 저장
    image = sitk.GetImageFromArray(data)
    image.SetSpacing(np.asarray(spacing, dtype=np.float))
    return image

#sitk_image -> data
def sitk_image_to_data(image):
    #data에 이미지Array값을 저장
    data = sitk.GetArrayFromImage(image)
    #3차원이면, 다음과 같이 회전
    if len(data.shape) == 3:
        data = np.rot90(data, -1, axes=(0, 2))
    return data


In [40]:
#unet3d/utils/utils.py
#이미지 파일들을 resize해서, 이미지들이나, 이미지 파일을 읽어오는 함수 정의

import pickle
import os
import collections

import nibabel as nib
import numpy as np
from nilearn.image import reorder_img, new_img_like

#얘는 모르겠다.... 별로 건드리고 싶지도 않고, 건드릴 필요도 없을듯, shape형태를 교정해주는것인듯
def fix_shape(image):
    if image.shape[-1] == 1:
        return image.__class__(dataobj=np.squeeze(image.get_data()), affine=image.affine)
    return image

#resize(이미지, 원하는 shape, 선형보간법)
def resize(image, new_shape, interpolation="linear"):
    #아핀 대각선이 있는 이미지를 반환, 함수에서 반환되는 새로운 이미지의 방향은 RAS(Right, Anterior, Superior로 정의)
    #너무 수학적인 내용이 들어갔다....
    image = reorder_img(image, resample=interpolation)
    #zoom 배율
    zoom_level = np.divide(new_shape, image.shape)
    #image.header.get_zooms()는 image의 복셀 크기를 mm단위로 얻는 함수, (2.0, 2.0, 2.19999..., 2000.0) 이런식으로 나온다는데,
    #마지막 값은, 카메라 스캔간격을 표시한다고 한다. 스캔간격은 (miliseconds)
    new_spacing = np.divide(image.header.get_zooms(), zoom_level)
    #resample_to_spacing의 정의는 위의것을 참조, (image의 데이터, image의 복셀크기, newspacing, 보간법)
    new_data = resample_to_spacing(image.get_data(), image.header.get_zooms(), new_spacing,
                                   interpolation=interpolation)
    #새 affine공간에 image공간 복사(affine공간이라고
    #기하학에서, 아핀 공간은 유클리드 공간의 아핀 기하학적 성질들을 일반화해서 만들어지는 구조이다.
    #핀 공간에서는 점에서 점을 빼서 벡터를 얻거나 점에 벡터를 더해 다른 점을 얻을 수는 있지만 원점이 없으므로 
    #점과 점을 더할 수는 없다. 이렇게 나와있는데, 알듯말듯 한데, 체감이 잘 안옴
    new_affine = np.copy(image.affine)
    
    #tolist는 배열을 목록으로 바꿔주는 함수
    np.fill_diagonal(new_affine, new_spacing.tolist() + [1])
    
    #new_affine공간의 좌표를 보정
    new_affine[:3, 3] += calculate_origin_offset(new_spacing, image.header.get_zooms())
    
    #참조 이미지와 동일한 클래스의 새 이미지를 만들어 반환, (참조이미지, 만들 이미지에 저장될 데이터, affin=new_affine행렬 적용)
    return new_img_like(image, new_data, affine=new_affine)

#이미지를 읽는함수, (파일, image_shape=None, 선형보간법, 추출)
def read_image(in_file, image_shape=None, interpolation='linear', crop=None):
    print("Reading: {0}".format(in_file))
    #파일 로드
    image = nib.load(in_file) # J.Lee: nib.load(os.path.abspath(in_file)) -> nib.load(in_file)
    image = fix_shape(image)
#     if crop:
#         image = crop_img_to(image, crop, copy=True)
    #image_shape가 true, 즉, shape값이 있으면, resize함수 적용후 반환, 아니면, 그냥 반환
    if image_shape:
        return resize(image, new_shape=image_shape, interpolation=interpolation)
    else:
        return image

#이미지 파일들을 읽는 함수

#함수에 들어있는 파라미터 의미는 다음과 같다.
#image_files : 이미지파일들 경로
#image_shape : 이미지 형태, 가로*세로*높이 = (240, 240, 155)
#crop : 추출여부
#label_indices : 예상되는 라벨이미지의 인덱스(1, 2, 4)
#선형보간법을 사용하면, 레이블이 엉망이 되니, 건들지말라고 써있다!

def read_image_files(image_files, image_shape=None, crop=None, label_indices=None):
    """
    :param image_files: # J.Lee: paths of image_files.
    :param image_shape: # J.Lee: (d,l,m)(155,240,240)
    # J.Lee:
    :param label_indices:
        len(image_files)-1. i.e. expected index of label(truth) image 
        (when called through: write_image_data_to_file-> reslice_image-> read_image_files)
    :param crop: 
    :param use_nearest_for_last_file: If True, will use nearest neighbor interpolation for the last file. This is used
    because the last file may be the labels file. Using linear interpolation here would mess up the labels.
    :return: 
    """
    if label_indices is None:
        label_indices = []
    elif not isinstance(label_indices, collections.Iterable) or isinstance(label_indices, str):
        label_indices = [label_indices]
    image_list = list()
    for index, image_file in enumerate(image_files):
        if (label_indices is None and (index + 1) == len(image_files)) \
                or (label_indices is not None and index in label_indices):
            interpolation = "nearest"
        else:
            interpolation = "linear"
        image_list.append(read_image(image_file, image_shape=image_shape, crop=crop, interpolation=interpolation))

    return image_list


In [41]:
#unet3d/normalize.py
#슬라이스와 정규화 함수 정의

import os

import numpy as np
from nilearn.image import new_img_like

# from unet3d.utils.utils import resize, read_image_files
# from .utils import crop_img, crop_img_to, read_image #!!!Missing in .utils!!!

# def get_cropping_parameters(in_files):
#     if len(in_files) > 1:
#         foreground = get_complete_foreground(in_files)
#     else:
#         foreground = get_foreground_from_set_of_files(in_files[0], return_image=True)
#     return crop_img(foreground, return_slices=True, copy=True)

#슬라이스
def reslice_image_set(in_files, image_shape, out_files=None, label_indices=None, crop=False):
    if crop:
        pass
#         crop_slices = get_cropping_parameters([in_files])
    else:
        crop_slices = None
    #이미지 파일들을 불러와서   
    images = read_image_files(in_files, image_shape=image_shape, crop=crop_slices, label_indices=label_indices)
    #Out_files가 None이 아닐경우, 즉, 존재할 경우, 다음을 실행
    if out_files:
        for image, out_file in zip(images, out_files):
            #모든 이미지마다, filename을 붙여줌, 즉, out_file과 매칭
            image.to_filename(out_file)
        return [out_file for out_file in out_files]
        # J.Lee: original: [os.path.abspath(out_file) for out_file in out_files]
    else:
        return images

#data 정규화
def normalize_data(data, mean, std):
    #newaxis는 간단히 말해, 차원을 늘려주는것이라고 생각하면 된다.
    #data = data - mean[:, np.newaxis, np.newaxis, np.newaxis]
    data -= mean[:, np.newaxis, np.newaxis, np.newaxis]
    #data = data / std[:, np.newaxis, np.newaxis, np.newaxis]
    data /= std[:, np.newaxis, np.newaxis, np.newaxis]
    return data

#정규화된 데이터를 저장
def normalize_data_storage(data_storage):
    #빈 리스트 생성
    means = list()
    stds = list()
    
    for index in range(data_storage.shape[0]):
        data = data_storage[index]
        means.append(data.mean(axis=(1, 2, 3)))
        stds.append(data.std(axis=(1, 2, 3)))
    mean = np.asarray(means).mean(axis=0)
    std = np.asarray(stds).mean(axis=0)
    for index in range(data_storage.shape[0]):
        data_storage[index] = normalize_data(data_storage[index], mean, std)
    return data_storage

In [42]:
#unet3d/data.py
#hdf5파일을 생성 함수와, 생성된 hdf5파일에 데이터셋 기록 및 추가, 읽어오는 함수가 정의된 부분인듯

import os

import numpy as np
import tables

#from .normalize import normalize_data_storage, reslice_image_set

#데이터 파일 생성 함수 정의
def create_data_file(out_file, n_channels, n_samples, image_shape):
    #hdf5파일 쓰기모드
    hdf5_file = tables.open_file(out_file, mode='w')
    #complevel : 필터속성의 컨테이너(데이터의 압축 수준을 지정, 허용되는 범위는 0-9, 값 0 (기본값)은 압축을 비활성화)
    #complib : 사용할 압축 라이브러리
    filters = tables.Filters(complevel=5, complib='blosc')
    data_shape = tuple([0, n_channels] + list(image_shape)) # J.Lee: (155,240,240) -> [0,4,155,240,240]
    truth_shape = tuple([0, 1] + list(image_shape)) # J.Lee: (155,240,240) -> [0,1,155,240,240]
    
    #딥러닝 훈련용, 하둡파일 생성 (본 코드에서는 hdf5로 생성)
    data_storage = hdf5_file.create_earray(hdf5_file.root, 'data', tables.Float32Atom(), shape=data_shape,
                                           filters=filters, expectedrows=n_samples)
    truth_storage = hdf5_file.create_earray(hdf5_file.root, 'truth', tables.UInt8Atom(), shape=truth_shape,
                                            filters=filters, expectedrows=n_samples)
    affine_storage = hdf5_file.create_earray(hdf5_file.root, 'affine', tables.Float32Atom(), shape=(0, 4, 4),
                                             filters=filters, expectedrows=n_samples)
    return hdf5_file, data_storage, truth_storage, affine_storage

#파일에 이미지 데이터 기록
def write_image_data_to_file(image_files, data_storage, truth_storage, image_shape, n_channels, affine_storage,
                             truth_dtype=np.uint8, crop=True):
    for set_of_files in image_files:
        # J.Lee: {set of files} includes paths of 1~4 channel images + 1truth image
        #슬라이스된 이미셋을 images에 저장
        images = reslice_image_set(set_of_files, image_shape, label_indices=len(set_of_files) - 1, crop=crop)
        #images에 저장된 이미지에서 data를 뽑아내, subject_data에 저장
        subject_data = [image.get_data() for image in images]
        #생덩된 하둡파일에 데이터 추가
        add_data_to_storage(data_storage, truth_storage, affine_storage, subject_data, images[0].affine, n_channels,
                            truth_dtype)
    return data_storage, truth_storage

#생성된 하둡파일에 데이터 추가
#그냥, 각각 세 list에 각각 추가하는것
def add_data_to_storage(data_storage, truth_storage, affine_storage, subject_data, affine, n_channels, truth_dtype):
    data_storage.append(np.asarray(subject_data[:n_channels])[np.newaxis])
    truth_storage.append(np.asarray(subject_data[n_channels], dtype=truth_dtype)[np.newaxis][np.newaxis])
    affine_storage.append(np.asarray(affine)[np.newaxis])


#hdf5 파일에 data 기록
#학습 이미지 세트를 가져와서 해당 이미지를 hdf5 파일에 기록한다.
#함수에 들어있는 파라미터의 의미는 다음과 같다.
# training_data_files : 훈련 데이터 파일을 포함하는 튜플 목록. modality(t1, t2, t1ce, flair)은 튜플내에서 다음과 같은 순서로 나열되어야한다.
# 마지막 last item은 각 튜플에서 라벨링된 이미지다. 그래서, 1을 빼준다.

#outfile : hdf5 파일이 쓰일 경로
#image_shape : hdf5파일에 저장될 이미지의 shape
#truth_dtype : default = 8-bit 양의 정수로 설정

def write_data_to_file(training_data_files, out_file, image_shape, truth_dtype=np.uint8, subject_ids=None,
                       normalize=True, crop=False): # J.Lee: crop=True -> crop=False
    """
    Takes in a set of training images and writes those images to an hdf5 file.
    :param training_data_files: List of tuples containing the training data files. The modalities should be listed in
    the same order in each tuple. The last item in each tuple must be the labeled image. 
    Example: [('sub1-T1.nii.gz', 'sub1-T2.nii.gz', 'sub1-truth.nii.gz'), 
              ('sub2-T1.nii.gz', 'sub2-T2.nii.gz', 'sub2-truth.nii.gz')]
    :param out_file: Where the hdf5 file will be written to.
    :param image_shape: Shape of the images that will be saved to the hdf5 file.
    :param truth_dtype: Default is 8-bit unsigned integer.
    :return: Location of the hdf5 file with the image data written to it. 
    """
    n_samples = len(training_data_files)
    n_channels = 4 #SSH: len(training_data_files[0]) - 1, t1, t1ce, t2, flair, label된 이미지[seg], 그래서, 1을 빼준다.
    
    #이상 없으면, 데이터 파일 생성 함수에 파라미터 값을 넘겨주고, 함수를 실행후, 리턴된 값을, 각각 파일에 저장
    try:
        hdf5_file, data_storage, truth_storage, affine_storage = create_data_file(out_file,
                                                                                  n_channels=n_channels,
                                                                                  n_samples=n_samples,
                                                                                  image_shape=image_shape)
    #예외 발생시, 미완성된 datafile을 삭제, 그래서 예전에 만들어놓은 hdf5파일이 없어진듯
    except Exception as e:
        # If something goes wrong, delete the incomplete data file
        os.remove(out_file)
        raise e

    #파일에 이미지 데이터 기록
    write_image_data_to_file(training_data_files, data_storage, truth_storage, image_shape,
                             truth_dtype=truth_dtype, n_channels=n_channels, affine_storage=affine_storage, crop=crop)
    #subject_ids값이 들어오면, hdf5_file에 subject_ids라는 
    if subject_ids:
        hdf5_file.create_array(hdf5_file.root, 'subject_ids', obj=subject_ids)
    #normalize값이 들어오면, datastorage 부분을 정규화 시키고, hdf5파일 닫기
    if normalize:
        normalize_data_storage(data_storage)
    hdf5_file.close()
    
    return out_file

#읽기모드로, 테이블파일 열기
def open_data_file(filename, readwrite="r"):
    return tables.open_file(filename, readwrite)


In [43]:
#modified from N.Freidman's preprocessing code
#나오미코드를 참조해서 만든 전처리
#우리가 가진 데이터 셋을 분류 및 원하는 순서대로 정렬하고, 트레이닝 데이터 파일에 저장
#hdf5파일을 생성 함수와, 생성된 hdf5파일에 데이터셋 기록 및 추가, 읽어오는 함수가 정의

data_dir = 'C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/'
DATA_HGG = data_dir+'HGG/'
DATA_LGG = data_dir+'LGG/'

#modality(mri 사진 종류를다음과 같은 순서로 정렬)
def modality_sorter(file_path, modality_order = ['flair', 't1','t1ce', 't2', 'seg']):
    '''used as key to sort {file path} by {modality_order}'''
    for i, modality in enumerate(modality_order):
        if modality in file_path: return i

#다음과 같이 빈배열을 만들고, BraTS19라는 이름이 경로내에 있으며, nii.gz라는 명을 가지고, nb4라는 명이 없으면, filepaths에 저장 
#아까 정의한 modality order대로, 정렬후,hgg_path와 lgg_path에 추가
hgg_paths = []
for dirpath, dirnames, files in os.walk(DATA_HGG):
    if ('BraTS19' in dirpath):
        file_paths = [dirpath+'/'+file for file in files if ('nii.gz' in file) and ('nb4' not in file)] # J.Lee: include N4Bias images if you want
        file_paths.sort(key=modality_sorter)
        hgg_paths.append(tuple(file_paths))

lgg_paths = []
for dirpath, dirnames, files in os.walk(DATA_LGG):
    if ('BraTS19' in dirpath):
        file_paths = [dirpath+'/'+file for file in files if ('nii.gz' in file) and ('nb4' not in file)] # J.Lee: include N4Bias images if you want
        file_paths.sort(key=modality_sorter)
        lgg_paths.append(tuple(file_paths))

training_data_files = hgg_paths+lgg_paths

#데이타 프레임을 보기 위한 코드
import pandas as pd
from IPython.display import display
pd.set_option('max_colwidth', 120)

display(pd.DataFrame(training_data_files[1]))
display(pd.DataFrame(training_data_files[-1]))

out_file = data_dir+'htf5/'
image_shape = (155,240,240)

,0
0,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_...
1,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_...
2,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_...
3,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_...
4,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_...


,0
0,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...
1,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...
2,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...
3,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...
4,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...


In [44]:
#위에서 정의한, write_data_to_file을 실행해, 해당 경로에, h5_outfiles.h5 파일 생성, 파일이 존재하면 생략해도 무방
'''
training_data_files = training_data_files
outfiles = 'C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/h5_outfiles.h5'
image_shape = (155,240,240)
write_data_to_file(training_data_files, outfiles, image_shape)
'''
# J.Lee comment: write_data_to_file function will be executed in the final step ('main() function')

"\ntraining_data_files = training_data_files\noutfiles = 'C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/h5_outfiles.h5'\nimage_shape = (155,240,240)\nwrite_data_to_file(training_data_files, outfiles, image_shape)\n"

In [45]:
#generator related functions

import tensorflow

#utils/utils.py
#pickle 함수 정의

#pickle write
def pickle_dump(item, out_file):
    with open(out_file, "wb") as opened_file:
        pickle.dump(item, opened_file)

#pickle load
def pickle_load(in_file):
    with open(in_file, "rb") as opened_file:
        return pickle.load(opened_file)

#utils/patches.py
# import numpy as np
#patch 인덱스 연산 및, 코너나 경계값에서, patch를 어떻게 처리할지에 대해 정의 된 함수가 들어있다.

#patch 인덱스 연산함수(image_shape, patch_size, overlap, start=None)
#image_shape : (155, 240, 240)
#patch_shape : 이미지 shape를 탐색하면서 훑어볼 작은 상자
#overlap : patch간의 겹치는부분
def compute_patch_indices(image_shape, patch_size, overlap, start=None):
    #overlap 변수가 int형이면, 실행하여,overlap값을 결정 
    if isinstance(overlap, int):
        #overlap = np.asarray([overlap]*len[155, 240, 240]), 다음과 같이 연산하여, 겹치는 부분을 결정
        overlap = np.asarray([overlap] * len(image_shape))
        
    #start가 None이면, 실행
    if start is None:
        #(image_shape/patch_size-overlap) 연산 후 올림하여, n_patches에 저장
        #즉, 패치들의 개수 = 전체 3D이미지/patch들간의 합집합 연산 후 반올림
        #그냥, patch들의 수 구하는 공식
        n_patches = np.ceil(image_shape / (patch_size - overlap))
        #(patch_size - overlap) * n_patches - image_shape + overlap 연산 후 overflow에 저장
        #즉, overflow는 patch들간의 겹치지 않은 부분들간의 집합 + 겹치는 부분들의 집합 - image 전체 모양
        overflow = (patch_size - overlap) * n_patches - image_shape + overlap
        #overflow/2한 값을 반올림 한후 -부호를 붙여, start 값으로 지정
        start = -np.ceil(overflow/2)
    #start 값이 정상적으로 정수값으로 존재하면
    elif isinstance(start, int):
        #다음과 같이 연산하여, 시작지점을 결정
        start = np.asarray([start] * len(image_shape))
    #stop은 탐색 종료부분
    stop = image_shape + start
    #step은 patch로 전체 이미지를 훑는데, 몇단계가 걸리느냐?
    step = patch_size - overlap
    #구한, start, stop, step함수로 get_set_of_patch_indices함수를 호출
    return get_set_of_patch_indices(start, stop, step)

#start, stop, step 값을 받아, 각 axis별로, 값을 세팅한 후에, 정수로 되어있는 전치행렬로 만든후, np.mgird로 차원뻥튀기를 해준다.
#만약, start가 0, stop이 10이고, step이 5면, 0, 2, 4, 6, 8값을 가지고, step이 2이면, 0, 5 이런 값을 가진다.
#step값 미지정시, 0, 1, 2, 3 .... 9까지 증가한다.
def get_set_of_patch_indices(start, stop, step):
    return np.asarray(np.mgrid[start[0]:stop[0]:step[0], start[1]:stop[1]:step[1],
                               start[2]:stop[2]:step[2]].reshape(3, -1).T, dtype=np.int)

#patch index를 임의로 얻어온다.
def get_random_patch_index(image_shape, patch_shape):
    #patch의 코너 인덱스를 랜덤하게 반환한다. 만약, training중 이 함수를 사용하면, 중간 픽셀이 가장자리 픽셀보다 더 자주 보이게 될것이다.
    #이건 나쁜 일이니, 트레이닝 중에 사용하지 말라는것 같다.
    """
    Returns a random corner index for a patch. If this is used during training, the middle pixels will be seen by
    the model way more often than the edge pixels (which is probably a bad thing).
    :param image_shape: Shape of the image
    :param patch_shape: Shape of the patch
    :return: a tuple containing the corner index which can be used to get a patch from an image
    """
    #image_shape와 patch_shape의 차를 구한후 랜덤하게 반환한다.
    return get_random_nd_index(np.subtract(image_shape, patch_shape))

#nd index를 임의로 얻어온다.
def get_random_nd_index(index_max):
    return tuple([np.random.choice(index_max[index] + 1) for index in range(len(index_max))])

#3d 데이터에서 patch를 얻어온다.

#data : patch에서 얻어온 numpy array
#patch_shape : patch의 모양/사이즈
#patch_index : patch의 코너인덱스
def get_patch_from_3d_data(data, patch_shape, patch_index):
    """
    Returns a patch from a numpy array.
    :param data: numpy array from which to get the patch.
    :param patch_shape: shape/size of the patch.
    :param patch_index: corner index of the patch.
    :return: numpy array take from the data with the patch shape specified.
    """
    patch_index = np.asarray(patch_index, dtype=np.int16)
    patch_shape = np.asarray(patch_shape)
    image_shape = data.shape[-3:]
    
    #np.any : np배열 내부에서 조건에 맞는 데이터가 있으면 True, 아니면, False 반환
    #즉, patch가 image의 범위에서 벗어난 부분에 대해서, 보정을 해주는 부분인듯, 즉, 작은 patch라는 상자가, 큰 상자의 바깥부분에
    #존재할때, 그 값들을 보정하기 위하여, fix_out_of_bound_patch_attempt함수를 불러와서, 보정해주는것인듯, 그 외에는 그냥 리턴
    if np.any(patch_index < 0) or np.any((patch_index + patch_shape) > image_shape):
        data, patch_index = fix_out_of_bound_patch_attempt(data, patch_shape, patch_index)
    return data[..., patch_index[0]:patch_index[0]+patch_shape[0], patch_index[1]:patch_index[1]+patch_shape[1],
                patch_index[2]:patch_index[2]+patch_shape[2]]

#경계부분 patch 수정, 이 함수는 위의, 3d_data로 부터, patch를 얻어올때만 쓰인다.
def fix_out_of_bound_patch_attempt(data, patch_shape, patch_index, ndim=3):
    """
    Pads the data and alters the patch index so that a patch will be correct.
    :param data:
    :param patch_shape:
    :param patch_index:
    :return: padded data, fixed patch index
    """
    #image_shape = data.shape
    image_shape = data.shape[-ndim:]
    #pad_before = patch가 음수면, 제곱해서, 절대값으로 변환
    pad_before = np.abs((patch_index < 0) * patch_index)
    #pad_after = patch가 image_shape를 이탈하면, 다음 방법으로 값을 보정
    pad_after = np.abs(((patch_index + patch_shape) > image_shape) * ((patch_index + patch_shape) - image_shape))
    #pad_before, pad_after를 axis = 1로 병합
    pad_args = np.stack([pad_before, pad_after], axis=1)
    #만약, pad_args의 차원이 
    #tolist는 nd array를 list로 바꾸어주는 함수
    if pad_args.shape[0] < len(data.shape):
        pad_args = [[0, 0]] * (len(data.shape) - pad_args.shape[0]) + pad_args.tolist()
    #np.pad(data, padding array, edge부분으로 처리)
    #즉, 데이터가 존재하고, padding에 edge라고 처리하는것이다.
    #예를 들어, data = [[1,2],[3,4]], pad_args = [[2, 2], [2, 2]], mode = "edge"면
    '''
    e e e e e e
    e e e e e e
    e e 1 2 e e
    e e 3 4 e e
    e e e e e e
    e e e e e e
    
    e = edge모드로 처리될 부분, 이런식으로 처리된다는 의미, 즉, 전체 이미지의 가장자리 부분을 처리하는 부분이다.
    '''
    
    
    data = np.pad(data, pad_args, mode="edge")
    patch_index += pad_before
    #data와 patch index 반환
    return data, patch_index

#patch 목록 및 해당 patch index에서 원래 모양의 배열을 재구성합니다. 겹치는 패치는 평균화

#patch들로부터 재설계
#patches : numpy array로 된 patch들의 목록
#patch_indicces : patch들의 목록에 해당하는 index 목록
#patch들로부터 추출한 array의 shape
#default값 : 결과데이터의 기본값, 만약, patch 적용이 완료되면, 이 값은 덮어씌어진다.


#함수는 정의는 되어있는데, 사용된곳이 없어서 보류
def reconstruct_from_patches(patches, patch_indices, data_shape, default_value=0):
    """
    Reconstructs an array of the original shape from the lists of patches and corresponding patch indices. Overlapping
    patches are averaged.
    :param patches: List of numpy array patches.
    :param patch_indices: List of indices that corresponds to the list of patches.
    :param data_shape: Shape of the array from which the patches were extracted.
    :param default_value: The default value of the resulting data. if the patch coverage is complete, this value will
    be overwritten.
    :return: numpy array containing the data reconstructed by the patches.
    """
    data = np.ones(data_shape) * default_value
    image_shape = data_shape[-3:]
    count = np.zeros(data_shape, dtype=np.int)
    for patch, index in zip(patches, patch_indices):
        image_patch_shape = patch.shape[-3:]
        if np.any(index < 0):
            fix_patch = np.asarray((index < 0) * np.abs(index), dtype=np.int)
            patch = patch[..., fix_patch[0]:, fix_patch[1]:, fix_patch[2]:]
            index[index < 0] = 0
        if np.any((index + image_patch_shape) >= image_shape):
            fix_patch = np.asarray(image_patch_shape - (((index + image_patch_shape) >= image_shape)
                                                        * ((index + image_patch_shape) - image_shape)), dtype=np.int)
            patch = patch[..., :fix_patch[0], :fix_patch[1], :fix_patch[2]]
        patch_index = np.zeros(data_shape, dtype=np.bool)
        patch_index[...,
                    index[0]:index[0]+patch.shape[-3],
                    index[1]:index[1]+patch.shape[-2],
                    index[2]:index[2]+patch.shape[-1]] = True
        patch_data = np.zeros(data_shape)
        patch_data[patch_index] = patch.flatten()

        new_data_index = np.logical_and(patch_index, np.logical_not(count > 0))
        data[new_data_index] = patch_data[new_data_index]

        averaged_data_index = np.logical_and(patch_index, count > 0)
        if np.any(averaged_data_index):
            data[averaged_data_index] = (data[averaged_data_index] * count[averaged_data_index] + patch_data[averaged_data_index]) / (count[averaged_data_index] + 1)
        count[patch_index] += 1
    return data


#######################
#augmentaion에 사용된 함수들, scale 변경, 뒤집기, transpose등과 관련된 함수들의 정의되어 있다.
#augment.py
import numpy as np
import nibabel as nib
from nilearn.image import new_img_like, resample_to_img
import random
import itertools

#이미지에 크기변환 후, affine 적용하는 함수
#new_img_like : 참조 이미지와 동일한 새로운 이미지를 만드는데 사용하는 함수, (이미지, data, affine 배열행렬)
def scale_image(image, scale_factor):
    scale_factor = np.asarray(scale_factor)
    new_affine = np.copy(image.affine)
    new_affine[:3, :3] = image.affine[:3, :3] * scale_factor
    new_affine[:, 3][:3] = image.affine[:, 3][:3] + (image.shape * np.diag(image.affine)[:3] * (1 - scale_factor)) / 2
    return new_img_like(image, data=image.get_data(), affine=new_affine)


#이미지와 축을 이용해, axis를 기준으로, 반전해주는 함수
def flip_image(image, axis):
    try:
        new_data = np.copy(image.get_data())
        for axis_index in axis:
            new_data = np.flip(new_data, axis=axis_index)
    except TypeError:
        new_data = np.flip(image.get_data(), axis=axis)
    return new_img_like(image, data=new_data)

#아래의 augmentation data 함수에 사용하기 위해, 정의된 함수
#len(truth.shape)값을 인자로 받게됨
def random_flip_dimensions(n_dimensions):
    axis = list()
    for dim in range(n_dimensions):
        #random_boolean을 형성한다. True or False의 값을 뱉어내게 되는데, 1이면, axis에 axis에 값을 추가한다.
        if random_boolean():
            axis.append(dim)
    #결과적으로, (truth.shape의 factor들을 무작위로 axis에 집어 넣게 된다.)
    return axis

#얘도, augmentation data 함수에 사용하기 위해, 정의된 함수
#무작위 표본 정규분표를 만든다. np.random.normal(location, size, scale)
def random_scale_factor(n_dim=3, mean=1, std=0.25):
    return np.random.normal(mean, std, n_dim)

#위의 random_flip_dimensions에 사용하기 위해 정의된 함수
def random_boolean():
    return np.random.choice([True, False])

#얘도, augmentation data 함수에 사용하기 위해, 정의된 함수
#distort_image(이미지, 반전 기준 축, scale_factor), random으로 flip_axis에 append 했기 때문에, 값이 None일 수도 있고, 있을 수도 있다.
#flip_axis 값이 있다면, 해당 axis를 기준으로 반전
#scale_factor도 마찬가지
#이 둘을 적용시켜, 왜곡된 이미지를 뱉어낸다.
def distort_image(image, flip_axis=None, scale_factor=None):
    if flip_axis:
        image = flip_image(image, flip_axis)
    if scale_factor is not None:
        image = scale_image(image, scale_factor)
    return image

#위의 정의된 함수들을 다 이 함수에 적용시켜서, 데이터를 augmentation 하는 함수
def augment_data(data, truth, affine, scale_deviation=None, flip=True):
    n_dim = len(truth.shape)
    if scale_deviation:
        scale_factor = random_scale_factor(n_dim, std=scale_deviation)
    else:
        scale_factor = None
    if flip:
        flip_axis = random_flip_dimensions(n_dim)
    else:
        flip_axis = None
    data_list = list()
    
    for data_index in range(data.shape[0]):
        image = get_image(data[data_index], affine)
        #datalist에 왜곡을 시킨 이미지를 재조정후에 저장, resample_to_image함수에 대한 설명은 아래에
        #resample_to_img(source_img, target_img, interpolation = 'continuous',copy = True ,order = 'F' ,clip = False ,fill_value = 0 , force_resample = False )
        data_list.append(resample_to_img(distort_image(image, flip_axis=flip_axis,
                                                       scale_factor=scale_factor), image,
                                         interpolation="continuous").get_data())
    #아래도 같음
    data = np.asarray(data_list)
    truth_image = get_image(truth, affine)
    truth_data = resample_to_img(distort_image(truth_image, flip_axis=flip_axis, scale_factor=scale_factor),
                                 truth_image, interpolation="nearest").get_data()
    return data, truth_data


#nib.Nifti1Image(new_data, img.affine, img.header[얘는 선택사항, 없어도 됨])
#이미지에 affine 적용후, 새로운 이미지를 생성하는 함수, 위의 augment_data함수에서 사용됨
def get_image(data, affine, nib_class=nib.Nifti1Image):
    return nib_class(dataobj=data, affine=affine)


#여기는 만들어놓고 안쓴 함수들, 3D를 회전시키고, 뒤집고 하는데 사용한 함수들 정의


#이 함수는 48 개의 고유한 회전을 나타내는 "키"세트를 반환합니다. 즉, 3D 회전 매트릭스를 반환
#각 세트의 항목은 튜플로 되어있다. ((rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose)
#예를들어 ((0, 1), 0, 1, 0, 1) 이것은, z축을 중심으로 90도 회전후, y축 기준으로 뒤집고, transpose 시킨다.
def generate_permutation_keys():
    """
    This function returns a set of "keys" that represent the 48 unique rotations &
    reflections of a 3D matrix.

    Each item of the set is a tuple:
    ((rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose)

    As an example, ((0, 1), 0, 1, 0, 1) represents a permutation in which the data is
    rotated 90 degrees around the z-axis, then reversed on the y-axis, and then
    transposed.

    48 unique rotations & reflections:
    https://en.wikipedia.org/wiki/Octahedral_symmetry#The_isometries_of_the_cube
    """
    #집합을 반환
    #itertools.product는 데카르트 곱을 의미
    #즉, itertools.combinations_with_replacement(range(2), 2) 와 range(2), range(2), range(2), range(2), range(2)간에 일어날 수 있는
    #모든 순열을 생성
    #원래는 itertools.product( ,repeat=n) 값을 정해주어야 하나, repeat 값이 없으므로, 그냥 1
    #즉 여기서는, 그냥,나열한것
    
    #ex)itertools.product("ABC", 2)
    #>>> AA, AB, AC, BA, BB, BC, CA, CB, CC
    
    #itertools.combinations_with_replacement은, itertools.product에서 AB와 BA를 같은것으로 보는것, 조합임
    
    #ex)itertools.combinations_with_replacement('ABC', 2)
    #>>> AA AB AC BB BC CC

    #즉, 얘를 풀면
    #itertools.combinations_with_replacement(range(2), 2) = 00, 01, 11 3가지
    #itertools.product([00, 01, 11], [0, 1], [0, 1], [0, 1], [0, 1])
    #[]에서 하나씩 추출해서, 순열을 만든다. 그러면, 3*2*2*2*2 = 48가지가 나온다.
    
    return set(itertools.product(
        itertools.combinations_with_replacement(range(2), 2), range(2), range(2), range(2), range(2)))

#위의 list(generate_permutation_keys())에서 생성된 48개중 하나를 골라 반환
def random_permutation_key():
    """
    Generates and randomly selects a permutation key. See the documentation for the
    "generate_permutation_keys" function.
    """
    return random.choice(list(generate_permutation_keys()))


#주어진 키의 특성에 따라 데이터를 바꿔준다. 인풋데이터 모양은 반드시, 다음 형태를 가져야 한다.(n_modalities, x, y, z)
#인풋 키는 튜플로 다음과 같이 구성되어 있다. Input key is a tuple: (rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose)
#예를 들어, ((0, 1), 0, 1, 0, 1) 이것은, z축으로 90도 회전하고, y축 기준으로 반전시키고, transpose한것이다.

def permute_data(data, key):
    """
    Permutes the given data according to the specification of the given key. Input data
    must be of shape (n_modalities, x, y, z).

    Input key is a tuple: (rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose)

    As an example, ((0, 1), 0, 1, 0, 1) represents a permutation in which the data is
    rotated 90 degrees around the z-axis, then reversed on the y-axis, and then
    transposed.
    """
    
    #아래는, 튜플의 값에 따라, daata를 바꿔주는것을 구현한 코드
    data = np.copy(data)
    (rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose = key

    if rotate_y != 0:
        data = np.rot90(data, rotate_y, axes=(1, 3))
    if rotate_z != 0:
        data = np.rot90(data, rotate_z, axes=(2, 3))
    if flip_x:
        data = data[:, ::-1]
    if flip_y:
        data = data[:, :, ::-1]
    if flip_z:
        data = data[:, :, :, ::-1]
    if transpose:
        for i in range(data.shape[0]):
            data[i] = data[i].T
    return data

#위에서 정의한, random_permutation_key() 값인 키값을 받아, permute_data 함수를 수행하여, 데이터 변환
#들어오는 데이터는 반드시, (n_modalities, x, y, z)의 형태를 가진, numpy array여야 할것
def random_permutation_x_y(x_data, y_data):
    """
    Performs random permutation on the data.
    :param x_data: numpy array containing the data. Data must be of shape (n_modalities, x, y, z).
    :param y_data: numpy array containing the data. Data must be of shape (n_modalities, x, y, z).
    :return: the permuted data
    """
    key = random_permutation_key()
    return permute_data(x_data, key), permute_data(y_data, key)

#permute_data(data, key) 함수를 적용하기 전으로 돌아감 
def reverse_permute_data(data, key):
    key = reverse_permutation_key(key)
    data = np.copy(data)
    (rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose = key

    if transpose:
        for i in range(data.shape[0]):
            data[i] = data[i].T
    if flip_z:
        data = data[:, :, :, ::-1]
    if flip_y:
        data = data[:, :, ::-1]
    if flip_x:
        data = data[:, ::-1]
    if rotate_z != 0:
        data = np.rot90(data, rotate_z, axes=(2, 3))
    if rotate_y != 0:
        data = np.rot90(data, rotate_y, axes=(1, 3))
    return data

#random_permutation_key()를 적용하기 전 상황으로 복구하는 키를 만드는 함수
def reverse_permutation_key(key):
    rotation = tuple([-rotate for rotate in key[0]])
    return rotation, key[1], key[2], key[3], key[4]


###############################################################################

#generator관련 함수가 정의되있다.

#generator.py
import os
import copy
from random import shuffle
import itertools

import numpy as np

# from .utils import pickle_dump, pickle_load
# from .utils.patches import compute_patch_indices, get_random_nd_index, get_patch_from_3d_data
# from .augment import augment_data, random_permutation_x_y

#training 과 validation generator 정의, 모델을 트레이닝 할때 사용

#data_file : 트레이닝에 사용할 data_file
#batch_size : 트레이닝 batchsize
#n_labels : 이진화된 레이블 수
#training_keys_file : 훈련 데이터의 인덱스 위치가 기록된 picklefile
#validation_keys_file : 검증 데이터의 인덱스 위치가 기록된 picklefile

#data_split : 훈련 데이터와 검증데이터 분리, 0.8 -> 훈련데이터 : 검증데이터 = 8:2
#overwrite : 덮어쓰기
#labels = 이미지 파일들 내부에서 정렬된 라벨값들을 가진, 리스트나 튜플이다. 리스트 또는 튜플의 길이는 n_labels의 값과 같다.
#augment = 만약, True로 하면, 트레이닝 이미지가 오버피팅을 방지하기 위해, 왜곡된다.

#augment_flip : 만약, augment 값이 True고, 얘도 True이면, 무작위로, x, y, z축으로 반전된다.
#augment_distortion_factor : 만약, augment 값이 True이면, 원본과 왜곡된 데이터값의 표준편차를 계산한다.(늘어나거나, 수축된 정도), None으로
#하거나, Farse 또는 0으로 하면, 이 방식으로, 데이터가 왜곡되지 않는다.
#patch_shape : generator와 함께 반환할 데이터 shape, 만약, None값이면, 전체 이미지 shape가 반환

#validation_patch_overlap : 검증 데이터에서 겹쳐질 픽셀 / 복셀 수(필요한 patch_shape는 None이 아님)
#training_patch_start_offset : 정수값이 들어있는 길이 3의 튜플로, 트레이닝 데이터가 무작위로 (0, 0, 0) ~ (training_patch_start_offset) 
#사이의 픽셀 수 만큼 오프셋된다.

#validation_batch_size : 검증 데이터의 batchsize
#skip_blank : 만약 True면, 이미지 또는 패치에서 0값을 가진부분은 data generator에서 생략한다.
#permute : 무작위로 데이터를 치환, 데이터는 반드시 3차원 큐브형태여야한다.


def get_training_and_validation_generators(data_file, batch_size, n_labels, training_keys_file, validation_keys_file,
                                           data_split=0.8, overwrite=False, labels=None, augment=False,
                                           augment_flip=True, augment_distortion_factor=0.25, patch_shape=None,
                                           validation_patch_overlap=0, training_patch_start_offset=None,
                                           validation_batch_size=None, skip_blank=True, permute=False):
    """
    Creates the training and validation generators that can be used when training the model.
    :param skip_blank: If True, any blank (all-zero) label images/patches will be skipped by the data generator.
    :param validation_batch_size: Batch size for the validation data.
    :param training_patch_start_offset: Tuple of length 3 containing integer values. Training data will randomly be
    offset by a number of pixels between (0, 0, 0) and the given tuple. (default is None)
    :param validation_patch_overlap: Number of pixels/voxels that will be overlapped in the validation data. (requires
    patch_shape to not be None)
    :param patch_shape: Shape of the data to return with the generator. If None, the whole image will be returned.
    (default is None)
    :param augment_flip: if True and augment is True, then the data will be randomly flipped along the x, y and z axis
    :param augment_distortion_factor: if augment is True, this determines the standard deviation from the original
    that the data will be distorted (in a stretching or shrinking fashion). Set to None, False, or 0 to prevent the
    augmentation from distorting the data in this way.
    :param augment: If True, training data will be distorted on the fly so as to avoid over-fitting.
    :param labels: List or tuple containing the ordered label values in the image files. The length of the list or tuple
    should be equal to the n_labels value.
    Example: (10, 25, 50)
    The data generator would then return binary truth arrays representing the labels 10, 25, and 30 in that order.
    :param data_file: hdf5 file to load the data from.
    :param batch_size: Size of the batches that the training generator will provide.
    :param n_labels: Number of binary labels.
    :param training_keys_file: Pickle file where the index locations of the training data will be stored.
    :param validation_keys_file: Pickle file where the index locations of the validation data will be stored.
    :param data_split: How the training and validation data will be split. 0 means all the data will be used for
    validation and none of it will be used for training. 1 means that all the data will be used for training and none
    will be used for validation. Default is 0.8 or 80%.
    :param overwrite: If set to True, previous files will be overwritten. The default mode is false, so that the
    training and validation splits won't be overwritten when rerunning model training.
    :param permute: will randomly permute the data (data must be 3D cube)
    :return: Training data generator, validation data generator, number of training steps, number of validation steps
    """
    #만약, 검증 데이터의 배치사이즈가 지정되지 않으면, validation_batch_size = batch_size
    if not validation_batch_size:
        validation_batch_size = batch_size
        
    #training_list와 validation_list로 분리
    training_list, validation_list = get_validation_split(data_file,
                                                          data_split=data_split,
                                                          overwrite=overwrite,
                                                          training_file=training_keys_file,
                                                          validation_file=validation_keys_file)

    #training genrator 생성
    training_generator = data_generator(data_file, training_list,
                                        batch_size=batch_size,
                                        n_labels=n_labels,
                                        labels=labels,
                                        augment=augment,
                                        augment_flip=augment_flip,
                                        augment_distortion_factor=augment_distortion_factor,
                                        patch_shape=patch_shape,
                                        patch_overlap=0,
                                        patch_start_offset=training_patch_start_offset,
                                        skip_blank=skip_blank,
                                        permute=permute)
    
    #validation generator 생성
    validation_generator = data_generator(data_file, validation_list,
                                          batch_size=validation_batch_size,
                                          n_labels=n_labels,
                                          labels=labels,
                                          patch_shape=patch_shape,
                                          patch_overlap=validation_patch_overlap,
                                          skip_blank=skip_blank)

    # Set the number of training and testing samples per epoch correctly
    # 에포크 당 트레이닝 및 테스트 샘플 수를 설정
    num_training_steps = get_number_of_steps(get_number_of_patches(data_file, training_list, patch_shape,
                                                                   skip_blank=skip_blank,
                                                                   patch_start_offset=training_patch_start_offset,
                                                                   patch_overlap=0), batch_size)
    print("Number of training steps: ", num_training_steps)

    num_validation_steps = get_number_of_steps(get_number_of_patches(data_file, validation_list, patch_shape,
                                                                     skip_blank=skip_blank,
                                                                     patch_overlap=validation_patch_overlap),
                                               validation_batch_size)
    print("Number of validation steps: ", num_validation_steps)

    return training_generator, validation_generator, num_training_steps, num_validation_steps


#위의 generator 생성 함수에서 사용, step 수를 구하는 함수
def get_number_of_steps(n_samples, batch_size):
    if n_samples <= batch_size:
        return n_samples
    elif np.remainder(n_samples, batch_size) == 0:
        return n_samples//batch_size
    else:
        return n_samples//batch_size + 1

#위의 generator 생성 함수에서 사용, training data와 validation data를 분리하는데 사용하는 함수
def get_validation_split(data_file, training_file, validation_file, data_split=0.8, overwrite=False):
    """
    Splits the data into the training andc validation indices list.
    :param data_file: pytables hdf5 data file
    :param training_file:
    :param validation_file:
    :param data_split:
    :param overwrite:
    :return:
    """
    #isensee_training_ids.plk 파일이 없으면 raining과 validation을 분리해서, 저장 후 반환, 있으면, 존재한, trainig_list, validation_list를
    #반환
    if overwrite or not os.path.exists(training_file):
        print("Creating validation split...")
        nb_samples = data_file.root.data.shape[0]
        sample_list = list(range(nb_samples))
        training_list, validation_list = split_list(sample_list, split=data_split)
        pickle_dump(training_list, training_file)
        pickle_dump(validation_list, validation_file)
        return training_list, validation_list
    else:
        print("Loading previous validation split...")
        return pickle_load(training_file), pickle_load(validation_file)

#위의 get_validation_split함수에서 사용, 트레이닝셋과 테스팅셋으로 분리하는데 사용
def split_list(input_list, split=0.8, shuffle_list=True):
    if shuffle_list:
        shuffle(input_list)
    n_training = int(len(input_list) * split)
    training = input_list[:n_training]
    testing = input_list[n_training:]
    return training, testing

#data generator 생성함수, 파라미터에 대한 설명은 위의 training, validation generator와 같다.
def data_generator(data_file, index_list, batch_size=1, n_labels=1, labels=None, augment=False, augment_flip=True,
                   augment_distortion_factor=0.25, patch_shape=None, patch_overlap=0, patch_start_offset=None,
                   shuffle_index_list=True, skip_blank=True, permute=False):
    orig_index_list = index_list
    while True:
        x_list = list()
        y_list = list()
        #patch_shape 값이 들어오면, patch의 인덱스 list를 생성하는 함수를 실행하고, patch index를 index_list값에 저장
        if patch_shape:
            index_list = create_patch_index_list(orig_index_list, data_file.root.data.shape[-3:], patch_shape,
                                                 patch_overlap, patch_start_offset)
        #patch_shape 값이 들어오지 않으면, 그냥, 들어온 값을 그대로 사용
        else:
            index_list = copy.copy(orig_index_list)
        #shuffle_index_list가 참이면, 항목을 섞어준다.
        if shuffle_index_list:
            shuffle(index_list)
        #index_list에 값이 존재하는동안
        while len(index_list) > 0:
            #맨 마지막 인덱스부터, 하나씩 빼서, index에 저장
            index = index_list.pop()
            #데이터 파일의 데이터를 feature과 target data 리스트들에 추가하는 함수
            add_data(x_list, y_list, data_file, index, augment=augment, augment_flip=augment_flip,
                     augment_distortion_factor=augment_distortion_factor, patch_shape=patch_shape,
                     skip_blank=skip_blank, permute=permute)
            if len(x_list) == batch_size or (len(index_list) == 0 and len(x_list) > 0):
                #yield는 generator를 반환한다는것 빼고, return과 비슷
                yield convert_data(x_list, y_list, n_labels=n_labels, labels=labels)
                x_list = list()
                y_list = list()

#패치들의 개수를 얻는다.
def get_number_of_patches(data_file, index_list, patch_shape=None, patch_overlap=0, patch_start_offset=None,
                          skip_blank=True):
    #만약, patch_shape값이 존재하면
    if patch_shape:
        #patch의 인덱스 list를 생성하는 함수를 실행하고 반환된, patch_index를 index_list에 넣는다.
        index_list = create_patch_index_list(index_list, data_file.root.data.shape[-3:], patch_shape, patch_overlap,
                                             patch_start_offset)
        count = 0
        
        for index in index_list:
            x_list = list()
            y_list = list()
            #데이터 파일의 데이터를 feature과 target data 리스트들에 추가하는 함수
            add_data(x_list, y_list, data_file, index, skip_blank=skip_blank, patch_shape=patch_shape)
            #데이터파일로부터, 추가될 데이터가 있으면
            if len(x_list) > 0:
                count += 1
                #얘가 패치 수가 된다.
        return count
    #patch_shape값이 안들어 오면
    else:
        #얘가 패치 수가 된다.
        return len(index_list)


#data_generator 함수와 get_number_of_patches함수에서 사용, patch의 인덱스 list를 생성하는 함수
def create_patch_index_list(index_list, image_shape, patch_shape, patch_overlap, patch_start_offset=None):
    patch_index = list()
    for index in index_list:
        #만약, patch_start_offset이 None이 아니면
        if patch_start_offset is not None:
            #랜덤으로 start offset값을 음수값으로 받아, random_start_offset 값에 저장하고,
            random_start_offset = np.negative(get_random_nd_index(patch_start_offset))
            #코너나, 경계값에서, patch를 어떻게 처리할지에 대해 정의된 compute_patch_indices 함수를 실행시키면,  get_set_of_patch_indices
            #함수를 return 하여, 차원 뻥튀기를 해준다.
            patches = compute_patch_indices(image_shape, patch_shape, overlap=patch_overlap, start=random_start_offset)
        #그게 아니라면, start=None값으로 실행한다.
        else:
            patches = compute_patch_indices(image_shape, patch_shape, overlap=patch_overlap)
        #patch_index에 다음을 추가
        patch_index.extend(itertools.product([index], patches))
        #patch_index를 반환
    return patch_index

#add_data(), def data_generator와 def get_number_of_patches에 사용되는 함수이다.
#데이터 파일의 데이터를 feature과 target data 리스트들에 추가하는 함수
# x_list : 데이터파일로부터 추가될 데이터가 있는 데이터 목록
# y_list : 데이터파일로부터 추가될 target 데이터가 있는 데이터 목록
# data_file : hdf5 데이터 파일
# index : 데이터로부터 추출한 데이터파일들의 인덱스
# augment : 만약, True면, 데이터들은 augmentation parameter(augment_flip과 augment_distortion_factor)에 따라 augment 된다. 즉, augment 사용여부
# augment_flip : 만약, True면, x, y, z축을 따라 랜덤하게 뒤집힌다.
# augment_distortion_factor : 만약, True면, 원본과의 표준편차를 구한다. None, False, 0으로 세팅하면, 이 방식으로 왜곡된 데이터가, 추가되는것을 막을 수 있다.
# permute : 무작위로 데이터를 치환, 데이터는 반드시 3차원 큐브형태여야한다.
def add_data(x_list, y_list, data_file, index, augment=False, augment_flip=False, augment_distortion_factor=0.25,
             patch_shape=False, skip_blank=True, permute=False):
    """
    Adds data from the data file to the given lists of feature and target data
    :param skip_blank: Data will not be added if the truth vector is all zeros (default is True).
    :param patch_shape: Shape of the patch to add to the data lists. If None, the whole image will be added.
    :param x_list: list of data to which data from the data_file will be appended.
    :param y_list: list of data to which the target data from the data_file will be appended.
    :param data_file: hdf5 data file.
    :param index: index of the data file from which to extract the data.
    :param augment: if True, data will be augmented according to the other augmentation parameters (augment_flip and
    augment_distortion_factor)
    :param augment_flip: if True and augment is True, then the data will be randomly flipped along the x, y and z axis
    :param augment_distortion_factor: if augment is True, this determines the standard deviation from the original
    that the data will be distorted (in a stretching or shrinking fashion). Set to None, False, or 0 to prevent the
    augmentation from distorting the data in this way.
    :param permute: will randomly permute the data (data must be 3D cube)
    :return:
    """
    data, truth = get_data_from_file(data_file, index, patch_shape=patch_shape)
    if augment:
        if patch_shape is not None:
            affine = data_file.root.affine[index[0]]
        else:
            affine = data_file.root.affine[index]
        data, truth = augment_data(data, truth, affine, flip=augment_flip, scale_deviation=augment_distortion_factor)

    if permute:
        if data.shape[-3] != data.shape[-2] or data.shape[-2] != data.shape[-1]:
            raise ValueError("To utilize permutations, data array must be in 3D cube shape with all dimensions having "
                             "the same length.")
        data, truth = random_permutation_x_y(data, truth[np.newaxis])
    else:
        truth = truth[np.newaxis]

    if not skip_blank or np.any(truth != 0):
        x_list.append(data)
        y_list.append(truth)

#위의 add_data에서 사용하기 위해, 정의된 함수, 파일로부터, 데이터를 얻어오는 함수
#쉽게 말해, patch_shape값의 여부에 따라, data_file로부터 데이터를 얻어, data와 truth으로 나누고
#그것을 다시, get_patch_from_3d_data에 넣어, patch값을 얻어, 반환하거나, patch_shape=None이면, data_file.root.data로부터 얻어서 반환
def get_data_from_file(data_file, index, patch_shape=None):
    #만약, patch_shape 값이 들어오면
    if patch_shape:
        #데이터로부터, 추출한 데이터 파일들의 인덱스, index, patch_index에 저장한다.
        index, patch_index = index
        #get_data_from_file을 실행해서, 얻은 값인, x, y값을 data와 truth에 넣어준다.
        data, truth = get_data_from_file(data_file, index, patch_shape=None)
        #x에 3d data에서 patch를 얻어온 값을 저장
        x = get_patch_from_3d_data(data, patch_shape, patch_index)
        #y에 3d truth에서 patch를 얻어온 값을 저장
        y = get_patch_from_3d_data(truth, patch_shape, patch_index)
        
        #patch_shape값이 들어오지 않으면
    else:
        x, y = data_file.root.data[index], data_file.root.truth[index, 0]
    return x, y

#data 변환
def convert_data(x_list, y_list, n_labels=1, labels=None):
    x = np.asarray(x_list)
    y = np.asarray(y_list)
    if n_labels == 1:
        y[y > 0] = 1
    elif n_labels > 1:
        y = get_multi_class_labels(y, n_labels=n_labels, labels=labels)
    return x, y

#convert_data 에서 사용되는 함수
#label map을 이진label들로 변환하기 위한 함수
#data : shape를 가진 label map을 포함하는 numpy 배열
#n_labels : label들의 수
#labels : label들의 정수값
#shape의 이진 numpy 배열
def get_multi_class_labels(data, n_labels, labels=None):
    """
    Translates a label map into a set of binary labels.
    :param data: numpy array containing the label map with shape: (n_samples, 1, ...).
    :param n_labels: number of labels.
    :param labels: integer values of the labels.
    :return: binary numpy array of shape: (n_samples, n_labels, ...)
    """
    #data와 label들의 수로, new_shape를 만든다.
    new_shape = [data.shape[0], n_labels] + list(data.shape[2:])
    #newshape의 크기만큼, zero array를 생성하여, 초기화 한다.
    y = np.zeros(new_shape, np.int8)
    #다음 조건을 만족하면, 해당 인덱스에 1값으로 대체한다.
    #라벨이 있는 부분, 즉, 병변이라고 판단되는 라벨에, 1값을 넣어주는 y array 생성
    for label_index in range(n_labels):
        if labels is not None:
            y[:, label_index][data[:, 0] == labels[label_index]] = 1
        else:
            y[:, label_index][data[:, 0] == (label_index + 1)] = 1
    return y


In [46]:
#metrics.py
#하나 이상의 인수가 이미 채워진 함수의 새 버전을 만들기 위해 사용되는 모듈
from functools import partial

from tensorflow.keras import backend as K

#dice 계수를 구하는 함수
def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = tf.cast(K.flatten(y_true), tf.float32, name=None)# K.flatten(y_true)
    y_pred_f = tf.cast(K.flatten(y_pred), tf.float32, name=None)
    #J.Lee: original: y_pred_f = K.flatten(y_pred)
    #to prevent Value error. original y_pred is int8 and cannot be multiplied with fp32(y_true)
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

#dice 계수 loss
def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)

#weighted_dice_coefficient를 구하는 함수
def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):
    """
    Weighted dice coefficient. Default axis assumes a "channels first" data structure
    :param smooth:
    :param y_true:
    :param y_pred:
    :param axis:
    :return:
    """
    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                               axis=axis) + smooth))

#weighted_dice_coefficient를 구하는 함수, 그냥 부호만 -
def weighted_dice_coefficient_loss(y_true, y_pred):
    return -weighted_dice_coefficient(y_true, y_pred)

#label_wise_dice_coefficeint
def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coefficient(y_true[:, label_index], y_pred[:, label_index])


def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f


dice_coef = dice_coefficient
dice_coef_loss = dice_coefficient_loss


#training.py

import math
from functools import partial

from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model

# from unet3d.metrics import (dice_coefficient, dice_coefficient_loss, dice_coef, dice_coef_loss,
#                             weighted_dice_coefficient_loss, weighted_dice_coefficient)

K.set_image_data_format('channels_first')
# J.Lee: K.set_image_dim_ordering('th') -> K.set_image_data_format('channels_first') for use in keras ver2.x


# learning rate schedule
def step_decay(epoch, initial_lrate, drop, epochs_drop):
    return initial_lrate * math.pow(drop, math.floor((1+epoch)/float(epochs_drop)))


def get_callbacks(model_file, initial_learning_rate=0.0001, learning_rate_drop=0.5, learning_rate_epochs=None,
                  learning_rate_patience=50,
                  logging_file= data_dir+"training.log", #J.Lee:original:logging_file="training.log"
                  verbosity=1, early_stopping_patience=None):
    callbacks = list()
    callbacks.append(ModelCheckpoint(model_file, monitor='loss', #J.Lee:original: monitor='val_loss'(default)
                                     save_best_only=True))
    callbacks.append(CSVLogger(logging_file, append=True))
    if learning_rate_epochs:
        callbacks.append(LearningRateScheduler(partial(step_decay, initial_lrate=initial_learning_rate,
                                                       drop=learning_rate_drop, epochs_drop=learning_rate_epochs)))
    else:
        callbacks.append(ReduceLROnPlateau(factor=learning_rate_drop, patience=learning_rate_patience,
                                           verbose=verbosity))
    if early_stopping_patience:
        callbacks.append(EarlyStopping(verbose=verbosity, patience=early_stopping_patience))
    return callbacks


def load_old_model(model_file):
    print("Loading pre-trained model")
    custom_objects = {'dice_coefficient_loss': dice_coefficient_loss, 'dice_coefficient': dice_coefficient,
                      'dice_coef': dice_coef, 'dice_coef_loss': dice_coef_loss,
                      'weighted_dice_coefficient': weighted_dice_coefficient,
                      'weighted_dice_coefficient_loss': weighted_dice_coefficient_loss}
    try:
        from keras_contrib.layers import InstanceNormalization
        custom_objects["InstanceNormalization"] = InstanceNormalization
    except ImportError:
        pass
    try:
        return load_model(model_file, custom_objects=custom_objects)
    except ValueError as error:
        if 'InstanceNormalization' in str(error):
            raise ValueError(str(error) + "\n\nPlease install keras-contrib to use InstanceNormalization:\n"
                                          "'pip install git+https://www.github.com/keras-team/keras-contrib.git'")
        else:
            raise error


def train_model(model, model_file, training_generator, validation_generator, steps_per_epoch, validation_steps,
                initial_learning_rate=0.001, learning_rate_drop=0.5, learning_rate_epochs=None, n_epochs=500,
                learning_rate_patience=20, early_stopping_patience=None):
    """
    Train a Keras model.
    :param early_stopping_patience: If set, training will end early if the validation loss does not improve after the
    specified number of epochs.
    :param learning_rate_patience: If learning_rate_epochs is not set, the learning rate will decrease if the validation
    loss does not improve after the specified number of epochs. (default is 20)
    :param model: Keras model that will be trained.
    :param model_file: Where to save the Keras model.
    :param training_generator: Generator that iterates through the training data.
    :param validation_generator: Generator that iterates through the validation data.
    :param steps_per_epoch: Number of batches that the training generator will provide during a given epoch.
    :param validation_steps: Number of batches that the validation generator will provide during a given epoch.
    :param initial_learning_rate: Learning rate at the beginning of training.
    :param learning_rate_drop: How much at which to the learning rate will decay.
    :param learning_rate_epochs: Number of epochs after which the learning rate will drop.
    :param n_epochs: Total number of epochs to train the model.
    :return: 
    """
    model.fit(training_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=n_epochs,
                    validation_data=None, #J.LEE :original: validation_generator,
                    validation_steps=validation_steps,
                    callbacks=get_callbacks(model_file,
                                            initial_learning_rate=initial_learning_rate,
                                            learning_rate_drop=learning_rate_drop,
                                            learning_rate_epochs=learning_rate_epochs,
                                            learning_rate_patience=learning_rate_patience,
                                            early_stopping_patience=early_stopping_patience))
# J.Lee:original:
#     model.fit_generator(generator=training_generator,
#                         steps_per_epoch=steps_per_epoch,
#                         epochs=n_epochs,
#                         validation_data=validation_generator,
#                         validation_steps=validation_steps,
#                         callbacks=get_callbacks(model_file,
#                                                 initial_learning_rate=initial_learning_rate,
#                                                 learning_rate_drop=learning_rate_drop,
#                                                 learning_rate_epochs=learning_rate_epochs,
#                                                 learning_rate_patience=learning_rate_patience,
#                                                 early_stopping_patience=early_stopping_patience))


In [47]:
#model/unet.py

import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model #J.Lee:original_code: from keras.engine import Input, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose
#J.Lee:original_code: Deconvolution3D instead of Conv3DTranspose
from tensorflow.keras.optimizers import Adam

#import tensorflow_addons as tfa #J.Lee: extracode added for InstanceNormalization 

# from unet3d.metrics import dice_coefficient_loss, get_label_dice_coefficient_function, dice_coefficient

K.set_image_data_format("channels_first")

try:
    from keras.engine import merge
except ImportError:
    from tensorflow.keras.layers import concatenate 
    #J.Lee:original_code: from keras.layers.merge import concatenate 


# def unet_model_3d(input_shape, pool_size=(2, 2, 2), n_labels=1, initial_learning_rate=0.00001, deconvolution=False,
#                   depth=4, n_base_filters=32, include_label_wise_dice_coefficients=False, metrics=dice_coefficient,
#                   batch_normalization=False, activation_name="sigmoid"):
#     """
#     Builds the 3D UNet Keras model.f
#     :param metrics: List metrics to be calculated during model training (default is dice coefficient).
#     :param include_label_wise_dice_coefficients: If True and n_labels is greater than 1, model will report the dice
#     coefficient for each label as metric.
#     :param n_base_filters: The number of filters that the first layer in the convolution network will have. Following
#     layers will contain a multiple of this number. Lowering this number will likely reduce the amount of memory required
#     to train the model.
#     :param depth: indicates the depth of the U-shape for the model. The greater the depth, the more max pooling
#     layers will be added to the model. Lowering the depth may reduce the amount of memory required for training.
#     :param input_shape: Shape of the input data (n_chanels, x_size, y_size, z_size). The x, y, and z sizes must be
#     divisible by the pool size to the power of the depth of the UNet, that is pool_size^depth.
#     :param pool_size: Pool size for the max pooling operations.
#     :param n_labels: Number of binary labels that the model is learning.
#     :param initial_learning_rate: Initial learning rate for the model. This will be decayed during training.
#     :param deconvolution: If set to True, will use transpose convolution(deconvolution) instead of up-sampling. This
#     increases the amount memory required during training.
#     :return: Untrained 3D UNet Model
#     """
#     inputs = Input(input_shape)
#     current_layer = inputs
#     levels = list()

#     # add levels with max pooling
#     for layer_depth in range(depth):
#         layer1 = create_convolution_block(input_layer=current_layer, n_filters=n_base_filters*(2**layer_depth),
#                                           batch_normalization=batch_normalization)
#         layer2 = create_convolution_block(input_layer=layer1, n_filters=n_base_filters*(2**layer_depth)*2,
#                                           batch_normalization=batch_normalization)
#         if layer_depth < depth - 1:
#             current_layer = MaxPooling3D(pool_size=pool_size)(layer2)
#             levels.append([layer1, layer2, current_layer])
#         else:
#             current_layer = layer2
#             levels.append([layer1, layer2])

#     # add levels with up-convolution or up-sampling
#     for layer_depth in range(depth-2, -1, -1):
#         up_convolution = get_up_convolution(pool_size=pool_size, deconvolution=deconvolution,
#                                             #J.Lee:original: n_filters=current_layer._keras_shape[1]
#                                             n_filters= tensorflow.keras.backend.int_shape(current_layer)[1])(current_layer)
#         concat = concatenate([up_convolution, levels[layer_depth][1]], axis=1)
#         current_layer = create_convolution_block(n_filters= tensorflow.keras.backend.int_shape(levels[layer_depth][1])[1],
#                                                  # J.Lee:original: levels[layer_depth][1]._keras_shape[1],
#                                                  input_layer=concat, batch_normalization=batch_normalization)
#         current_layer = create_convolution_block(n_filters= tensorflow.keras.backend.int_shape(levels[layer_depth][1])[1],
#                                                  # J.Lee:original: levels[layer_depth][1]._keras_shape[1],
#                                                  input_layer=current_layer,
#                                                  batch_normalization=batch_normalization)

#     final_convolution = Conv3D(n_labels, (1, 1, 1))(current_layer)
#     act = Activation(activation_name)(final_convolution)
#     model = Model(inputs=inputs, outputs=act)

#     if not isinstance(metrics, list):
#         metrics = [metrics]

#     if include_label_wise_dice_coefficients and n_labels > 1:
#         label_wise_dice_metrics = [get_label_dice_coefficient_function(index) for index in range(n_labels)]
#         if metrics:
#             metrics = metrics + label_wise_dice_metrics
#         else:
#             metrics = label_wise_dice_metrics

#     model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss=dice_coefficient_loss, metrics=metrics)
#     return model


def create_convolution_block(input_layer, n_filters, batch_normalization=False, kernel=(3, 3, 3), activation=None,
                             padding='same', strides=(1, 1, 1), instance_normalization=False):
    """

    :param strides:
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides)(input_layer)
    if batch_normalization:
        layer = BatchNormalization(axis=1)(layer)
    elif instance_normalization:
        try:
            #J.Lee:original: from keras_contrib.layers.normalization import InstanceNormalization
            #from keras_contrib.layers.normalization import InstanceNormalization
            from tensorflow_addons.layers import InstanceNormalization
        except ImportError:
            raise ImportError("Install keras_contrib in order to use instance normalization."
                              "\nTry: pip install git+https://www.github.com/farizrahman4u/keras-contrib.git")
        layer = InstanceNormalization(axis=1)(layer)
    if activation is None:
        return Activation('relu')(layer)
    else:
        return activation()(layer)


def compute_level_output_shape(n_filters, depth, pool_size, image_shape):
    """
    Each level has a particular output shape based on the number of filters used in that level and the depth or number 
    of max pooling operations that have been done on the data at that point.
    :param image_shape: shape of the 3d image.
    :param pool_size: the pool_size parameter used in the max pooling operation.
    :param n_filters: Number of filters used by the last node in a given level.
    :param depth: The number of levels down in the U-shaped model a given node is.
    :return: 5D vector of the shape of the output node 
    """
    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Conv3DTranspose(filters=n_filters, kernel_size=kernel_size,
                               strides=strides) # J.Lee:original:Deconvolution3D
    else:
        return UpSampling3D(size=pool_size)

    
## isensee2017.py
from functools import partial

from tensorflow.keras.layers import Input, LeakyReLU, Add, UpSampling3D, Activation, SpatialDropout3D, Conv3D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

# from .unet import create_convolution_block, concatenate
# from ..metrics import weighted_dice_coefficient_loss


create_convolution_block = partial(create_convolution_block, activation=LeakyReLU, instance_normalization=True)


def isensee2017_model(input_shape=(4, 128, 128, 128), n_base_filters=16, depth=5, dropout_rate=0.3,
                      n_segmentation_levels=3, n_labels=4, optimizer=Adam, initial_learning_rate=5e-4,
                      loss_function=weighted_dice_coefficient_loss, activation_name="sigmoid"):
    """
    This function builds a model proposed by Isensee et al. for the BRATS 2017 competition:
    https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/MICCAI_BraTS_2017_proceedings_shortPapers.pdf
    This network is highly similar to the model proposed by Kayalibay et al. "CNN-based Segmentation of Medical
    Imaging Data", 2017: https://arxiv.org/pdf/1701.03056.pdf
    :param input_shape:
    :param n_base_filters:
    :param depth:
    :param dropout_rate:
    :param n_segmentation_levels:
    :param n_labels:
    :param optimizer:
    :param initial_learning_rate:
    :param loss_function:
    :param activation_name:
    :return:
    """
    inputs = Input(input_shape)

    current_layer = inputs
    level_output_layers = list()
    level_filters = list()
    for level_number in range(depth):
        n_level_filters = (2**level_number) * n_base_filters
        level_filters.append(n_level_filters)

        if current_layer is inputs:
            in_conv = create_convolution_block(current_layer, n_level_filters)
        else:
            in_conv = create_convolution_block(current_layer, n_level_filters, strides=(2, 2, 2))

        context_output_layer = create_context_module(in_conv, n_level_filters, dropout_rate=dropout_rate)

        summation_layer = Add()([in_conv, context_output_layer])
        level_output_layers.append(summation_layer)
        current_layer = summation_layer

    segmentation_layers = list()
    for level_number in range(depth - 2, -1, -1):
        up_sampling = create_up_sampling_module(current_layer, level_filters[level_number])
        concatenation_layer = concatenate([level_output_layers[level_number], up_sampling], axis=1)
        localization_output = create_localization_module(concatenation_layer, level_filters[level_number])
        current_layer = localization_output
        if level_number < n_segmentation_levels:
            segmentation_layers.insert(0, Conv3D(n_labels, (1, 1, 1))(current_layer))

    output_layer = None
    for level_number in reversed(range(n_segmentation_levels)):
        segmentation_layer = segmentation_layers[level_number]
        if output_layer is None:
            output_layer = segmentation_layer
        else:
            output_layer = Add()([output_layer, segmentation_layer])

        if level_number > 0:
            output_layer = UpSampling3D(size=(2, 2, 2))(output_layer)

    activation_block = Activation(activation_name)(output_layer)

    model = Model(inputs=inputs, outputs=activation_block)
    model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function)
    return model


def create_localization_module(input_layer, n_filters):
    convolution1 = create_convolution_block(input_layer, n_filters)
    convolution2 = create_convolution_block(convolution1, n_filters, kernel=(1, 1, 1))
    return convolution2


def create_up_sampling_module(input_layer, n_filters, size=(2, 2, 2)):
    up_sample = UpSampling3D(size=size)(input_layer)
    convolution = create_convolution_block(up_sample, n_filters)
    return convolution


def create_context_module(input_layer, n_level_filters, dropout_rate=0.3, data_format="channels_first"):
    convolution1 = create_convolution_block(input_layer=input_layer, n_filters=n_level_filters)
    dropout = SpatialDropout3D(rate=dropout_rate, data_format=data_format)(convolution1)
    convolution2 = create_convolution_block(input_layer=dropout, n_filters=n_level_filters)
    return convolution2


In [48]:
K.set_image_data_format("channels_first")

In [49]:
#This code substitute 'fetch_training_data_files'
#aware that modality_order[:4] is equivalant to config["all_modalities"]

#modified from N.Freidman's preprocessing code
data_dir = 'C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/'
DATA_HGG = data_dir+'HGG/'
DATA_LGG = data_dir+'LGG/'

def modality_sorter(file_path, modality_order = ['flair', 't1','t1ce', 't2', 'seg']):
    '''used as key to sort {file path} by {modality_order}'''
    for i, modality in enumerate(modality_order):
        if modality in file_path: return i

hgg_paths = []
hgg_subject_ids = []
for dirpath, dirnames, files in os.walk(DATA_HGG):
    if len(dirnames)>0 :hgg_subject_ids=dirnames
    if ('BraTS19' in dirpath):
        file_paths = [dirpath+'/'+file for file in files if ('nii.gz' in file) and ('nb4' not in file)] # J.Lee: include N4Bias images if you want
        file_paths.sort(key=modality_sorter)
        hgg_paths.append(tuple(file_paths))

lgg_paths = []
lgg_subject_ids = []
for dirpath, dirnames, files in os.walk(DATA_LGG):
    if len(dirnames)>0 :lgg_subject_ids=dirnames
    if ('BraTS19' in dirpath):
        file_paths = [dirpath+'/'+file for file in files if ('nii.gz' in file) and ('nb4' not in file)] # J.Lee: include N4Bias images if you want
        file_paths.sort(key=modality_sorter)
        lgg_paths.append(tuple(file_paths))

training_files = hgg_paths+lgg_paths
subject_ids = hgg_subject_ids+lgg_subject_ids
        
import pandas as pd
from IPython.display import display
pd.set_option('max_colwidth', 120)

display(pd.DataFrame(training_files[5]))
print(subject_ids[5])
display(pd.DataFrame(training_files[-1]))
print(subject_ids[-1])
print(len(training_files), len(subject_ids))

,0
0,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_17_1/BraTS19_...
1,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_17_1/BraTS19_...
2,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_17_1/BraTS19_...
3,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_17_1/BraTS19_...
4,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_17_1/BraTS19_...


BraTS19_2013_17_1


,0
0,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...
1,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...
2,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...
3,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...
4,C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS1...


BraTS19_TMC_09043_1
335 335


In [50]:
import os
import glob

# from unet3d.data import write_data_to_file, open_data_file
# from unet3d.generator import get_training_and_validation_generators
# from unet3d.model import isensee2017_model
# from unet3d.training import load_old_model, train_model


config = dict()
config["image_shape"] = (128, 128, 128)  # This determines what shape the images will be cropped/resampled to.
config["patch_shape"] = None  # switch to None to train on the whole image
config["labels"] = (1, 2, 4)  # the label numbers on the input image
config["n_base_filters"] = 16
config["n_labels"] = len(config["labels"])
config["all_modalities"] = ['flair', 't1','t1ce', 't2'] #J.Lee:original:["t1", "t1ce", "flair", "t2"]
config["training_modalities"] = config["all_modalities"]  # change this if you want to only use some of the modalities
config["nb_channels"] = len(config["training_modalities"])
if "patch_shape" in config and config["patch_shape"] is not None:
    config["input_shape"] = tuple([config["nb_channels"]] + list(config["patch_shape"]))
else:
    config["input_shape"] = tuple([config["nb_channels"]] + list(config["image_shape"]))
config["truth_channel"] = config["nb_channels"]
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["batch_size"] = 1
config["validation_batch_size"] = 2
config["n_epochs"] = 10 #500  # cutoff the training after this many epochs
config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 50  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 5e-4
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["validation_split"] = 0.8  # portion of the data that will be used for training
config["flip"] = False  # augments the data by randomly flipping an axis during
config["permute"] = True  # data shape must be a cube. Augments the data by permuting in various directions
config["distort"] = None  # switch to None if you want no distortion
config["augment"] = config["flip"] or config["distort"]
config["validation_patch_overlap"] = 0  # if > 0, during training, validation patches will be overlapping
config["training_patch_start_offset"] = (16, 16, 16)  # randomly offset the first patch index by up to this offset
config["skip_blank"] = True  # if True, then patches without any target will be skipped

data_dir = 'C:/Users/user/Desktop/MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/'
config["data_file"] = data_dir+ "brats_data.h5"#os.path.abspath("brats_data.h5")
config["model_file"] = data_dir+ "isensee_2017_model.h5"#os.path.abspath("isensee_2017_model.h5")
config["training_file"] = data_dir+ "isensee_training_ids.pkl" #os.path.abspath("isensee_training_ids.pkl")
config["validation_file"] = data_dir+ "isensee_validation_ids.pkl"#os.path.abspath("isensee_validation_ids.pkl")
config["overwrite"] = False  # If True, will previous files. If False, will use previously written files.



# def fetch_training_data_files(return_subject_ids=False):
#     training_data_files = list()
#     subject_ids = list()
#     for subject_dir in glob.glob(os.path.join(os.path.dirname(__file__), "data", "preprocessed", "*", "*")):
#         subject_ids.append(os.path.basename(subject_dir))
#         subject_files = list()
#         for modality in config["training_modalities"] + ["truth"]:
#             subject_files.append(os.path.join(subject_dir, modality + ".nii.gz"))
#         training_data_files.append(tuple(subject_files))
#     if return_subject_ids:
#         return training_data_files, subject_ids
#     else:
#         return training_data_files

#J.Lee:comments on fetch_training_data_files
#Naomi 코드의 os.walk로 만든것과 동일한 기능 (단, preprocess.py 로 처리 해놓은 데이터라고 가정하는듯;)
#config에 적힌 값을 가지고 리스트안의 튜플을 만들어옴
#BraTS19_2013_2_1/BraTS19_2013_2_1_t1_nb4.nii.gz 일때
#폴더명인 BraTS19_2013_2_1은 subject_ids 리스트에 담기고
#전체 절대경로인 C:/MICCAI_BraTS_2019_Data_Training/HGG/CBraTS19_2013_2_1/BraTS19_2013_2_1_t1_nb4.nii.gz
# training_data_files리스트의 원소인 튜플의 원소로 들어감.

overwrite=config["overwrite"]
# convert input images into an hdf5 file

if overwrite or not os.path.exists(config["data_file"]):
    #J.Lee: training_files, subject_ids = fetch_training_data_files(return_subject_ids=True)
    write_data_to_file(training_files, config["data_file"], image_shape=config["image_shape"],
                       subject_ids=subject_ids)
data_file_opened = open_data_file(config["data_file"])

if not overwrite and os.path.exists(config["model_file"]):
    model = load_old_model(config["model_file"])
else:
    # instantiate new model
    model = isensee2017_model(input_shape=config["input_shape"], n_labels=config["n_labels"],
                              initial_learning_rate=config["initial_learning_rate"],
                              n_base_filters=config["n_base_filters"])

In [51]:
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 4, 128, 128, 0                                            
__________________________________________________________________________________________________
conv3d_60 (Conv3D)              (None, 16, 128, 128, 1744        input_3[0][0]                    
__________________________________________________________________________________________________
instance_normalization_54 (Inst (None, 16, 128, 128, 32          conv3d_60[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_54 (LeakyReLU)      (None, 16, 128, 128, 0           instance_normalization_54[0][0]  
____________________________________________________________________________________________

In [52]:
# Checking layers and their index before training
for layer,i in zip(model.layers,range(0,len(model.layers))):
    print(f"{i: <3}",
          f"{layer.name[:20]: <20} ",
          f"{str(layer.output_shape[1:5]): <20}",
          f"{layer.count_params(): <8}", #f"{np.array(layer.get_weights()).size: <8}",
          f"{layer.trainable}",
          layer.dtype)
    
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

0   input_3               []                   0        True float32
1   conv3d_60             (16, 128, 128, 128)  1744     True float32
2   instance_normalizati  (16, 128, 128, 128)  32       True float32
3   leaky_re_lu_54        (16, 128, 128, 128)  0        True float32
4   conv3d_61             (16, 128, 128, 128)  6928     True float32
5   instance_normalizati  (16, 128, 128, 128)  32       True float32
6   leaky_re_lu_55        (16, 128, 128, 128)  0        True float32
7   spatial_dropout3d_10  (16, 128, 128, 128)  0        True float32
8   conv3d_62             (16, 128, 128, 128)  6928     True float32
9   instance_normalizati  (16, 128, 128, 128)  32       True float32
10  leaky_re_lu_56        (16, 128, 128, 128)  0        True float32
11  add_14                (16, 128, 128, 128)  0        True float32
12  conv3d_63             (32, 64, 64, 64)     13856    True float32
13  instance_normalizati  (32, 64, 64, 64)     64       True float32
14  leaky_re_lu_57        (32, 64,

In [53]:
# get training and testing generators
train_generator, validation_generator, n_train_steps, n_validation_steps = get_training_and_validation_generators(
    data_file_opened,
    batch_size=config["batch_size"],
    data_split=config["validation_split"],
    overwrite=overwrite,
    validation_keys_file=config["validation_file"],
    training_keys_file=config["training_file"],
    n_labels=config["n_labels"],
    labels=config["labels"],
    patch_shape=config["patch_shape"],
    validation_batch_size=config["validation_batch_size"],
    validation_patch_overlap=config["validation_patch_overlap"],
    training_patch_start_offset=config["training_patch_start_offset"],
    permute=config["permute"],
    augment=config["augment"],
    skip_blank=config["skip_blank"],
    augment_flip=config["flip"],
    augment_distortion_factor=config["distort"])


Creating validation split...
Number of training steps:  268
Number of validation steps:  34


In [54]:
# run training

# run training
train_model(model=model,
            model_file=config["model_file"],
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=n_train_steps,
            validation_steps=n_validation_steps,
            initial_learning_rate=config["initial_learning_rate"],
            learning_rate_drop=config["learning_rate_drop"],
            learning_rate_patience=config["patience"],
            early_stopping_patience=config["early_stop"],
            n_epochs=10)#J.Lee:original: n_epochs=config["n_epochs"])
#data_file_opened.close()

Epoch 1/10


InvalidArgumentError:  Conv3DBackpropInputOpV2 only supports NDHWC on the CPU.
	 [[node gradient_tape/model_2/conv3d_89/Conv3DBackpropInputV2 (defined at <ipython-input-46-f7f6653b7673>:146) ]] [Op:__inference_train_function_82851]

Function call stack:
train_function


In [ ]:
# n_train_steps 조절해서 validation 넘어가는거 좀 더 쉽게 확인해보기.
# train_steps = 50
# validation_steps = 20 정도만 해도 되지 않을까;

#임비멘토님께 generator pickle화 관련 꼼수 있는지 여쭤보기.

In [ ]:
#data_file_opened.close()